In [1]:
import numpy as np
import os
import networkx as nx
from scipy.ndimage import zoom
def load_and_validate_data(file_path):
    """
    Load data from a file and return it.
    Return None if data is missing.
    """
    try:
        loaded_data = np.load(file_path)
        
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def combine_data_from_folder(folder_path, batch_size=100):
    """
    Combine data from all valid files in the folder in batches.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        combined_ppg.append(ppg_f)
        combined_ecg.append(ecg_f)
        combined_seg_dbp.append(seg_dbp)
        combined_seg_sbp.append(seg_sbp)
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.concatenate(combined_ppg, axis=0)
            combined_ecg = np.concatenate(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.concatenate(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.concatenate(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Validation_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'

In [2]:
val_data_generator = combine_data_from_folder(val_dir, batch_size=10)

In [3]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import os

vg_image_dir = 'Val_VG'


def load_vg_images(batch_idx):
    vg_images_path = os.path.join(vg_image_dir, f'Val_VG_batch_{batch_idx}.npz')
    
    if not os.path.exists(vg_images_path) :
        print(f"VG image files do not exist for Batch {batch_idx}.")
        return None
    
    vg_images = np.load(vg_images_path)['vg_images']

    
    return vg_images

def plot_vg_images(vg_images, num_images=5, image_size=(224, 224)):
    for i in range(min(num_images, vg_images.shape[0])):
        plt.figure(figsize=(5, 5))
        # Reshape each VG image to its original 2D shape
        img = vg_images[i].reshape(image_size)
        plt.imshow(img)
        plt.title(f'VG Image {i+1}')
        plt.axis('off')
        plt.show()


In [4]:

all_vg_images = []
all_sbp_values = []
all_dbp_values = []


for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(val_data_generator):
   
    
    if batch_idx + 1>3:
        break
    print(f"Processing Batch {batch_idx + 1}...")
    vg_images = load_vg_images(batch_idx + 1)
    if vg_images is None:
        print(f"No VG images found for Batch {batch_idx + 1}. Stopping...")
        break
    
    flattened_vg_images = np.array(vg_images)
    
    all_vg_images.append(flattened_vg_images)
    all_sbp_values.append(np.array(combined_seg_sbp_batch.flatten()))
    all_dbp_values.append(np.array(combined_seg_dbp_batch.flatten()))

    print(f"Batch {batch_idx + 1} processing complete.")
    

all_vg_images = np.concatenate(all_vg_images, axis=0)
all_sbp_values = np.concatenate(all_sbp_values, axis=0)
all_dbp_values = np.concatenate(all_dbp_values, axis=0)     

Processing Batch 1...
Batch 1 processing complete.
Processing Batch 2...
Batch 2 processing complete.
Processing Batch 3...
Batch 3 processing complete.


In [5]:
y = np.column_stack((all_sbp_values, all_dbp_values))
vg_images_normal = np.array(all_vg_images)

In [6]:
len(vg_images_normal)

14850

In [7]:
len(y)

14850

# VGG19

In [6]:
import numpy as np
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import Ridge




In [7]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

# Define the batch size
batch_size = 32

# Function to generate image batches
def image_generator(images, labels, batch_size):
    num_samples = images.shape[0]
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_images = images[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]
            batch_images_reshaped = batch_images.reshape(-1, 224, 224, 1)
            batch_images_rgb = np.concatenate([batch_images_reshaped] * 3, axis=-1)
            batch_images_preprocessed = preprocess_input(batch_images_rgb)
            yield batch_images_preprocessed, batch_labels


In [8]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = Flatten(name='flatten')(x)
#x = Dense(4096, activation='relu', name='fc1')(x)
#x = Dense(4096, activation='relu', name='fc2')(x)
output = Dense(512, activation='linear',name='fc1')(x)  


model = Model(inputs=base_model.input, outputs=output)


model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
images = vg_images_normal
labels = y
num_samples = images.shape[0]

In [ ]:
# Define a new model to extract features from the 'fc1' layer of the trained model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('fc1').output)

# Function to extract features
def extract_features(generator, num_samples):
    features = []
    for batch_images, _ in generator:
        batch_features = feature_extractor.predict(batch_images)
        features.append(batch_features)
        if len(features) * batch_size >= num_samples:
            break
    return np.vstack(features)


feature_generator_normal = image_generator(images, np.zeros((num_samples,)), batch_size)


features_normal = extract_features(feature_generator_normal, num_samples)


features_normal_flattened= features_normal.reshape(features_normal.shape[0],-1)


features= features_normal_flattened

In [ ]:
from sklearn.model_selection import train_test_split

vg_features_train,vg_features_test,label_vgg_train,label_vgg_test=train_test_split(features,labels, test_size=0.2, random_state=42)
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(vg_features_train, label_vgg_train)


predicted_values_split = ridge_model.predict(vg_features_test)

In [ ]:
predicted_sbp =predicted_values_split[:, 0]  # Assuming the first column is SBP
predicted_dbp =predicted_values_split[:, 1]  # Assuming the second column is DBP

true_sbp = label_vgg_test[:, 0]  # True SBP values from q
true_dbp = label_vgg_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

In [ ]:
num_samples

# MOBILE NET

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import Ridge

# Define the batch size
batch_size = 32

# Function to generate image batches
def image_generator(images, labels, batch_size):
    num_samples = images.shape[0]
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_images = images[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]
            batch_images_reshaped = batch_images.reshape(-1, 224, 224, 1)
            batch_images_rgb = np.concatenate([batch_images_reshaped] * 3, axis=-1)
            batch_images_preprocessed = preprocess_input(batch_images_rgb)
            yield batch_images_preprocessed, batch_labels


In [9]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet import preprocess_input


base_model_mobile = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model_mobile.layers:
    layer.trainable = False


x = base_model_mobile.output
x = Flatten(name='flatten')(x)
output = Dense(512, activation='linear', name='fc1')(x)


model_mobile = Model(inputs=base_model_mobile.input, outputs=output)

model_mobile.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128   

In [10]:
images = vg_images_normal
labels = y
num_samples = images.shape[0]

In [11]:
# Define a new model to extract features from the 'fc1' layer of the trained model
feature_extractor_mobile = Model(inputs=model_mobile.input, outputs=model_mobile.get_layer('fc1').output)

# Function to extract features
def extract_features(generator, num_samples):
    features = []
    for batch_images, _ in generator:
        batch_features = feature_extractor_mobile.predict(batch_images)
        features.append(batch_features)
        if len(features) * batch_size >= num_samples:
            break
    return np.vstack(features)


feature_generator_normal_1 = image_generator(images, np.zeros((num_samples,)), batch_size)


features_normal_1 = extract_features(feature_generator_normal_1, num_samples)

1/1 [==============================] - 1s 771ms/step


In [12]:
features_normal_flattened_1= features_normal_1.reshape(features_normal_1.shape[0],-1)


features_1= features_normal_flattened_1

In [13]:
from sklearn.model_selection import train_test_split

Mobile_train,Mobile_test,label_train,label_test=train_test_split(features_1,labels, test_size=0.2, random_state=42)
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(Mobile_train, label_train)


predicted_mobile = ridge_model.predict(Mobile_test)

In [14]:
predicted_sbp =predicted_mobile[:, 0]  # Assuming the first column is SBP
predicted_dbp =predicted_mobile[:, 1]  # Assuming the second column is DBP

true_sbp = label_test[:, 0]  # True SBP values from q
true_dbp = label_test[:, 1]  # True DBP values from q

def calculate_metrics(predicted, true):
    # Calculate R (correlation coefficient)
    R = np.corrcoef(predicted, true)[0, 1]

    # Calculate MAE (Mean Absolute Error)
    MAE = np.mean(np.abs(predicted - true))

    # Calculate RMSE (Root Mean Squared Error)
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    # Calculate ME ± SD (Mean Error ± Standard Deviation)
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)

# Calculate metrics for SBP
sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")

# Calculate metrics for DBP
dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")

SBP Metrics: R=0.5350883479323466, MAE=11.582464008512448, RMSE=15.454890610537387, ME±SD=(-0.2674706525288677, 15.452575941690514)
DBP Metrics: R=0.5773094399159031, MAE=6.686779229214582, RMSE=9.007300820469364, ME±SD=(-0.1878863144349513, 9.005341015379491)


In [15]:
num_samples

14850